In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.optim import optimizer
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
class LinearRegression(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)
    
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, input, target):
        return torch.sqrt(torch.mean((input - target) ** 2))

In [3]:
def load_data(folder_path):
  load_df = pd.DataFrame()
  # 遍歷文件夾中的所有文件
  name_list=[]
  for filename in os.listdir(folder_path):
      if filename.endswith(".csv"):
          file_path = os.path.join(folder_path, filename)
          df = pd.read_csv(file_path)
          load_df = pd.concat([load_df, df["現鈔買入"], df["現鈔賣出"], df["即期買入"], df["即期賣出"]], axis=1)
          filename = filename.replace(".csv", "")
          name_list.extend([f"{filename}現鈔買入", f"{filename}現鈔賣出", f"{filename}即期買入", f"{filename}即期賣出"])
  load_df.columns = name_list

  # 反向排序
  load_df = load_df.iloc[::-1]

  # 排序column
  load_df = load_df.reindex(sorted(load_df.columns), axis=1)

  # 處理空值
  load_df.replace("-", 0, inplace=True)

  return load_df

train_df = load_data("./train")
train_df

,AUD即期買入,AUD即期賣出,AUD現鈔買入,AUD現鈔賣出,CAD即期買入,CAD即期賣出,CAD現鈔買入,CAD現鈔賣出,EUR即期買入,EUR即期賣出,...,JPY現鈔買入,JPY現鈔賣出,KRW即期買入,KRW即期賣出,KRW現鈔買入,KRW現鈔賣出,USD即期買入,USD即期賣出,USD現鈔買入,USD現鈔賣出
3409,23.95,24.15,23.69,24.43,27.41,27.61,27.10,27.90,38.56,38.96,...,0.2706,0.2811,0,0,0.03037,0.03508,31.870,31.970,31.570,32.112
3408,24.00,24.20,23.74,24.49,27.52,27.72,27.21,28.01,38.62,39.02,...,0.2705,0.2810,0,0,0.03022,0.03491,31.930,32.030,31.630,32.172
3407,24.05,24.25,23.79,24.54,27.51,27.71,27.20,28.00,38.73,39.13,...,0.2703,0.2808,0,0,0.03026,0.03495,32.070,32.170,31.770,32.312
3406,23.97,24.17,23.71,24.46,27.32,27.52,27.01,27.80,38.64,39.04,...,0.2702,0.2807,0,0,0.03049,0.03522,32.085,32.185,31.785,32.327
3405,23.87,24.07,23.61,24.35,27.48,27.68,27.17,27.97,38.61,39.01,...,0.2702,0.2807,0,0,0.03043,0.03515,32.070,32.170,31.770,32.312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,21.00,21.23,20.73,21.51,23.44,23.66,23.05,23.96,34.26,34.66,...,0.2860,0.2988,0,0,0.02416,0.02806,31.325,31.425,30.975,31.645
3,21.00,21.23,20.73,21.51,23.42,23.64,23.03,23.94,34.18,34.58,...,0.2872,0.3000,0,0,0.02411,0.02801,31.370,31.470,31.020,31.690
2,21.15,21.38,20.88,21.66,23.44,23.66,23.05,23.96,34.27,34.67,...,0.2858,0.2986,0,0,0.02428,0.02818,31.300,31.400,30.950,31.620
1,21.19,21.42,20.92,21.70,23.56,23.78,23.17,24.08,34.33,34.73,...,0.2844,0.2972,0,0,0.02430,0.02820,31.220,31.320,30.870,31.540


In [4]:
# 設定輸入資料的天數範圍
input_date_data_size = 4
#要到第幾項
power=6
# 設定 seed
torch.manual_seed(1234)
np.random.seed(1234)

In [5]:
train = train_df.to_numpy()
train_size, feature_size = train.shape
# 以一段時間的資料當作輸入，故資料數量要扣掉輸入天數範圍
train_size = train_size - input_date_data_size

train_x = np.empty([train_size, feature_size * input_date_data_size*power], dtype = float)#乘power讓資料集擴展到次方項
train_y = np.empty([train_size, feature_size], dtype = float)

for idx in range(train_size):
    temp_data = np.array([])
    for count in range(input_date_data_size):
        temp_data = np.hstack([temp_data, train[idx + count]])#把前幾天的資料合併
        for nth_term in range(2,power+1):
            temp_data = np.hstack([temp_data, train[idx + count]**nth_term])#把前幾天的資料合併
    train_x[idx, :] = temp_data
    train_y[idx, :] = train[idx + input_date_data_size]#應該要預測到的實際第幾天資料

    # y值只留下現鈔買入
filtered_columns = [train_df.columns.get_loc(col) for col in train_df.columns if '現鈔買入' in col]
train_y = train_y[:, filtered_columns]

In [6]:
#Feature scaling
mean_x = np.mean(train_x, axis = 0)
std_x = np.std(train_x, axis = 0)
for i in range(len(train_x)):
    for j in range(len(train_x[0])):
        if std_x[j] != 0:
            train_x[i][j] = (train_x[i][j] - mean_x[j]) / std_x[j]

In [7]:
split_ratio = 0.2

# 計算驗證集的大小
val_size = int(train_size * split_ratio)

# 隨機生成索引以選擇要包含在驗證集中的樣本
indices = np.random.permutation(train_size)

# 使用索引切分數據
val_indices = indices[:val_size]
train_indices = indices[val_size:]

# 創建驗證集
val_x = train_x[val_indices]
val_y = train_y[val_indices]
# 創建訓練集
train_x = train_x[train_indices]
train_y = train_y[train_indices]

In [8]:
#轉為pytorch張量，才可以餵進pytorch中
train_x = torch.from_numpy(train_x.astype(np.float32))
train_y = torch.from_numpy(train_y.astype(np.float32))
val_x = torch.from_numpy(val_x.astype(np.float32))
val_y = torch.from_numpy(val_y.astype(np.float32))


In [9]:
model = LinearRegression(feature_size * input_date_data_size*power, 8)

best_validation_loss = float('inf')

learning_rates =0.01
criterion =RMSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rates)
patience = 10 
counter = 0  

In [10]:
epochs = 100000
val_loss_history = []
train_loss_history=[]
for epoch in range(epochs):
    model.train()
    # forward pass and loss
    train_y_predicted = model(train_x)
    loss = criterion(train_y_predicted, train_y)
    train_loss_history.append(loss.item())
    # backward pass
    loss.backward()
    # update
    optimizer.step()
    # init optimizer
    optimizer.zero_grad()

    # 驗證資料集
    model.eval()
    val_y_predicted = model(val_x)
    val_loss = criterion(val_y_predicted, val_y)
    val_loss_history.append(val_loss.item())
    
    if val_loss < best_validation_loss:
        best_validation_loss = val_loss
        counter = 0  # 重置计数器
    else:
        counter += 1
    if counter >= patience:
        break
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, train_loss = {loss.item(): .4f}, val_loss = {val_loss.item(): .4f}')

epoch: 10, train_loss =  28.0058, val_loss =  28.0248
epoch: 20, train_loss =  27.9596, val_loss =  27.9778
epoch: 30, train_loss =  27.9211, val_loss =  27.9388
epoch: 40, train_loss =  27.8887, val_loss =  27.9059
epoch: 50, train_loss =  27.8608, val_loss =  27.8776
epoch: 60, train_loss =  27.8364, val_loss =  27.8530
epoch: 70, train_loss =  27.8146, val_loss =  27.8312
epoch: 80, train_loss =  27.7948, val_loss =  27.8115
epoch: 90, train_loss =  27.7766, val_loss =  27.7935
epoch: 100, train_loss =  27.7595, val_loss =  27.7768
epoch: 110, train_loss =  27.7434, val_loss =  27.7611
epoch: 120, train_loss =  27.7279, val_loss =  27.7462
epoch: 130, train_loss =  27.7130, val_loss =  27.7318
epoch: 140, train_loss =  27.6985, val_loss =  27.7180
epoch: 150, train_loss =  27.6844, val_loss =  27.7045
epoch: 160, train_loss =  27.6705, val_loss =  27.6913
epoch: 170, train_loss =  27.6568, val_loss =  27.6783
epoch: 180, train_loss =  27.6433, val_loss =  27.6656
epoch: 190, train_l

epoch: 1550, train_loss =  25.9144, val_loss =  25.9732
epoch: 1560, train_loss =  25.9019, val_loss =  25.9607
epoch: 1570, train_loss =  25.8894, val_loss =  25.9483
epoch: 1580, train_loss =  25.8769, val_loss =  25.9358
epoch: 1590, train_loss =  25.8644, val_loss =  25.9234
epoch: 1600, train_loss =  25.8518, val_loss =  25.9109
epoch: 1610, train_loss =  25.8393, val_loss =  25.8984
epoch: 1620, train_loss =  25.8268, val_loss =  25.8860
epoch: 1630, train_loss =  25.8143, val_loss =  25.8735
epoch: 1640, train_loss =  25.8018, val_loss =  25.8611
epoch: 1650, train_loss =  25.7893, val_loss =  25.8486
epoch: 1660, train_loss =  25.7767, val_loss =  25.8361
epoch: 1670, train_loss =  25.7642, val_loss =  25.8237
epoch: 1680, train_loss =  25.7517, val_loss =  25.8112
epoch: 1690, train_loss =  25.7392, val_loss =  25.7988
epoch: 1700, train_loss =  25.7267, val_loss =  25.7863
epoch: 1710, train_loss =  25.7142, val_loss =  25.7738
epoch: 1720, train_loss =  25.7017, val_loss =  

epoch: 3210, train_loss =  23.8380, val_loss =  23.9009
epoch: 3220, train_loss =  23.8255, val_loss =  23.8884
epoch: 3230, train_loss =  23.8130, val_loss =  23.8759
epoch: 3240, train_loss =  23.8005, val_loss =  23.8634
epoch: 3250, train_loss =  23.7880, val_loss =  23.8509
epoch: 3260, train_loss =  23.7755, val_loss =  23.8384
epoch: 3270, train_loss =  23.7630, val_loss =  23.8259
epoch: 3280, train_loss =  23.7505, val_loss =  23.8134
epoch: 3290, train_loss =  23.7380, val_loss =  23.8009
epoch: 3300, train_loss =  23.7255, val_loss =  23.7884
epoch: 3310, train_loss =  23.7130, val_loss =  23.7759
epoch: 3320, train_loss =  23.7005, val_loss =  23.7634
epoch: 3330, train_loss =  23.6880, val_loss =  23.7509
epoch: 3340, train_loss =  23.6755, val_loss =  23.7384
epoch: 3350, train_loss =  23.6630, val_loss =  23.7259
epoch: 3360, train_loss =  23.6505, val_loss =  23.7134
epoch: 3370, train_loss =  23.6380, val_loss =  23.7009
epoch: 3380, train_loss =  23.6255, val_loss =  

epoch: 4780, train_loss =  21.8755, val_loss =  21.9381
epoch: 4790, train_loss =  21.8630, val_loss =  21.9256
epoch: 4800, train_loss =  21.8505, val_loss =  21.9131
epoch: 4810, train_loss =  21.8380, val_loss =  21.9006
epoch: 4820, train_loss =  21.8255, val_loss =  21.8881
epoch: 4830, train_loss =  21.8130, val_loss =  21.8756
epoch: 4840, train_loss =  21.8005, val_loss =  21.8631
epoch: 4850, train_loss =  21.7881, val_loss =  21.8506
epoch: 4860, train_loss =  21.7756, val_loss =  21.8381
epoch: 4870, train_loss =  21.7631, val_loss =  21.8256
epoch: 4880, train_loss =  21.7506, val_loss =  21.8131
epoch: 4890, train_loss =  21.7381, val_loss =  21.8006
epoch: 4900, train_loss =  21.7256, val_loss =  21.7881
epoch: 4910, train_loss =  21.7131, val_loss =  21.7756
epoch: 4920, train_loss =  21.7006, val_loss =  21.7631
epoch: 4930, train_loss =  21.6881, val_loss =  21.7505
epoch: 4940, train_loss =  21.6756, val_loss =  21.7380
epoch: 4950, train_loss =  21.6631, val_loss =  

epoch: 6310, train_loss =  19.9636, val_loss =  20.0249
epoch: 6320, train_loss =  19.9511, val_loss =  20.0124
epoch: 6330, train_loss =  19.9386, val_loss =  19.9999
epoch: 6340, train_loss =  19.9261, val_loss =  19.9874
epoch: 6350, train_loss =  19.9136, val_loss =  19.9748
epoch: 6360, train_loss =  19.9011, val_loss =  19.9623
epoch: 6370, train_loss =  19.8886, val_loss =  19.9498
epoch: 6380, train_loss =  19.8761, val_loss =  19.9373
epoch: 6390, train_loss =  19.8636, val_loss =  19.9248
epoch: 6400, train_loss =  19.8511, val_loss =  19.9123
epoch: 6410, train_loss =  19.8386, val_loss =  19.8998
epoch: 6420, train_loss =  19.8262, val_loss =  19.8873
epoch: 6430, train_loss =  19.8137, val_loss =  19.8748
epoch: 6440, train_loss =  19.8012, val_loss =  19.8623
epoch: 6450, train_loss =  19.7887, val_loss =  19.8498
epoch: 6460, train_loss =  19.7762, val_loss =  19.8373
epoch: 6470, train_loss =  19.7637, val_loss =  19.8248
epoch: 6480, train_loss =  19.7512, val_loss =  

...

epoch: 15500, train_loss =  8.4873, val_loss =  8.5294
epoch: 15510, train_loss =  8.4748, val_loss =  8.5169
epoch: 15520, train_loss =  8.4623, val_loss =  8.5043
epoch: 15530, train_loss =  8.4499, val_loss =  8.4918
epoch: 15540, train_loss =  8.4374, val_loss =  8.4793
epoch: 15550, train_loss =  8.4249, val_loss =  8.4668
epoch: 15560, train_loss =  8.4124, val_loss =  8.4543
epoch: 15570, train_loss =  8.4000, val_loss =  8.4418
epoch: 15580, train_loss =  8.3875, val_loss =  8.4293
epoch: 15590, train_loss =  8.3750, val_loss =  8.4168
epoch: 15600, train_loss =  8.3625, val_loss =  8.4043
epoch: 15610, train_loss =  8.3500, val_loss =  8.3918
epoch: 15620, train_loss =  8.3376, val_loss =  8.3792
epoch: 15630, train_loss =  8.3251, val_loss =  8.3667
epoch: 15640, train_loss =  8.3126, val_loss =  8.3542
epoch: 15650, train_loss =  8.3001, val_loss =  8.3417
epoch: 15660, train_loss =  8.2876, val_loss =  8.3292
epoch: 15670, train_loss =  8.2752, val_loss =  8.3167
epoch: 156

epoch: 17050, train_loss =  6.5532, val_loss =  6.5901
epoch: 17060, train_loss =  6.5407, val_loss =  6.5776
epoch: 17070, train_loss =  6.5282, val_loss =  6.5651
epoch: 17080, train_loss =  6.5157, val_loss =  6.5526
epoch: 17090, train_loss =  6.5033, val_loss =  6.5401
epoch: 17100, train_loss =  6.4908, val_loss =  6.5276
epoch: 17110, train_loss =  6.4783, val_loss =  6.5151
epoch: 17120, train_loss =  6.4658, val_loss =  6.5025
epoch: 17130, train_loss =  6.4534, val_loss =  6.4900
epoch: 17140, train_loss =  6.4409, val_loss =  6.4775
epoch: 17150, train_loss =  6.4284, val_loss =  6.4650
epoch: 17160, train_loss =  6.4159, val_loss =  6.4525
epoch: 17170, train_loss =  6.4034, val_loss =  6.4400
epoch: 17180, train_loss =  6.3910, val_loss =  6.4275
epoch: 17190, train_loss =  6.3785, val_loss =  6.4150
epoch: 17200, train_loss =  6.3660, val_loss =  6.4024
epoch: 17210, train_loss =  6.3535, val_loss =  6.3899
epoch: 17220, train_loss =  6.3411, val_loss =  6.3774
epoch: 172

epoch: 18590, train_loss =  4.6327, val_loss =  4.6639
epoch: 18600, train_loss =  4.6203, val_loss =  4.6514
epoch: 18610, train_loss =  4.6078, val_loss =  4.6389
epoch: 18620, train_loss =  4.5953, val_loss =  4.6264
epoch: 18630, train_loss =  4.5829, val_loss =  4.6139
epoch: 18640, train_loss =  4.5704, val_loss =  4.6014
epoch: 18650, train_loss =  4.5580, val_loss =  4.5889
epoch: 18660, train_loss =  4.5455, val_loss =  4.5764
epoch: 18670, train_loss =  4.5331, val_loss =  4.5639
epoch: 18680, train_loss =  4.5206, val_loss =  4.5514
epoch: 18690, train_loss =  4.5082, val_loss =  4.5389
epoch: 18700, train_loss =  4.4957, val_loss =  4.5265
epoch: 18710, train_loss =  4.4833, val_loss =  4.5140
epoch: 18720, train_loss =  4.4708, val_loss =  4.5015
epoch: 18730, train_loss =  4.4584, val_loss =  4.4890
epoch: 18740, train_loss =  4.4459, val_loss =  4.4765
epoch: 18750, train_loss =  4.4335, val_loss =  4.4640
epoch: 18760, train_loss =  4.4210, val_loss =  4.4515
epoch: 187

epoch: 20120, train_loss =  2.7295, val_loss =  2.7541
epoch: 20130, train_loss =  2.7171, val_loss =  2.7416
epoch: 20140, train_loss =  2.7047, val_loss =  2.7292
epoch: 20150, train_loss =  2.6923, val_loss =  2.7167
epoch: 20160, train_loss =  2.6799, val_loss =  2.7043
epoch: 20170, train_loss =  2.6675, val_loss =  2.6918
epoch: 20180, train_loss =  2.6551, val_loss =  2.6794
epoch: 20190, train_loss =  2.6427, val_loss =  2.6669
epoch: 20200, train_loss =  2.6303, val_loss =  2.6545
epoch: 20210, train_loss =  2.6179, val_loss =  2.6420
epoch: 20220, train_loss =  2.6055, val_loss =  2.6296
epoch: 20230, train_loss =  2.5931, val_loss =  2.6171
epoch: 20240, train_loss =  2.5807, val_loss =  2.6047
epoch: 20250, train_loss =  2.5683, val_loss =  2.5922
epoch: 20260, train_loss =  2.5559, val_loss =  2.5798
epoch: 20270, train_loss =  2.5435, val_loss =  2.5674
epoch: 20280, train_loss =  2.5311, val_loss =  2.5549
epoch: 20290, train_loss =  2.5187, val_loss =  2.5425
epoch: 203

epoch: 21740, train_loss =  0.7488, val_loss =  0.7642
epoch: 21750, train_loss =  0.7372, val_loss =  0.7525
epoch: 21760, train_loss =  0.7256, val_loss =  0.7408
epoch: 21770, train_loss =  0.7141, val_loss =  0.7292
epoch: 21780, train_loss =  0.7025, val_loss =  0.7176
epoch: 21790, train_loss =  0.6911, val_loss =  0.7061
epoch: 21800, train_loss =  0.6796, val_loss =  0.6945
epoch: 21810, train_loss =  0.6682, val_loss =  0.6830
epoch: 21820, train_loss =  0.6568, val_loss =  0.6716
epoch: 21830, train_loss =  0.6455, val_loss =  0.6601
epoch: 21840, train_loss =  0.6342, val_loss =  0.6488
epoch: 21850, train_loss =  0.6229, val_loss =  0.6374
epoch: 21860, train_loss =  0.6117, val_loss =  0.6261
epoch: 21870, train_loss =  0.6005, val_loss =  0.6148
epoch: 21880, train_loss =  0.5894, val_loss =  0.6036
epoch: 21890, train_loss =  0.5783, val_loss =  0.5924
epoch: 21900, train_loss =  0.5673, val_loss =  0.5813
epoch: 21910, train_loss =  0.5563, val_loss =  0.5702
epoch: 219

In [11]:
test_df = load_data("./test")

test_df

,AUD即期買入,AUD即期賣出,AUD現鈔買入,AUD現鈔賣出,CAD即期買入,CAD即期賣出,CAD現鈔買入,CAD現鈔賣出,EUR即期買入,EUR即期賣出,...,JPY現鈔買入,JPY現鈔賣出,KRW即期買入,KRW即期賣出,KRW現鈔買入,KRW現鈔賣出,USD即期買入,USD即期賣出,USD現鈔買入,USD現鈔賣出
3983,20.19,20.39,19.90,20.68,22.17,22.37,21.77,22.68,32.68,33.08,...,0.2223,0.2351,0,0,0.02165,0.02555,30.325,30.425,29.975,30.645
3982,20.21,20.41,19.92,20.70,22.28,22.48,21.88,22.79,32.81,33.21,...,0.2215,0.2343,0,0,0.02164,0.02554,30.405,30.505,30.055,30.725
3981,20.33,20.53,20.04,20.82,22.40,22.60,22.00,22.91,32.86,33.26,...,0.2207,0.2335,0,0,0.02171,0.02561,30.405,30.505,30.055,30.725
3980,20.23,20.43,19.94,20.72,22.37,22.57,21.97,22.88,32.95,33.35,...,0.2195,0.2323,0,0,0.02161,0.02551,30.400,30.500,30.050,30.720
3979,21.04,21.24,20.75,21.53,22.48,22.68,22.08,22.99,33.00,33.40,...,0.2418,0.2546,0,0,0.02281,0.02671,27.715,27.815,27.365,28.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,21.08,21.28,20.79,21.57,23.25,23.45,22.85,23.76,30.50,30.90,...,0.2154,0.2282,0,0,0.02116,0.02506,29.940,30.040,29.590,30.260
3,21.13,21.33,20.84,21.62,22.04,22.24,21.64,22.55,34.15,34.55,...,0.2675,0.2803,0,0,0.02432,0.02822,28.740,28.840,28.390,29.060
2,21.17,21.37,20.88,21.66,22.08,22.28,21.68,22.59,34.29,34.69,...,0.2675,0.2803,0,0,0.02428,0.02818,28.760,28.860,28.410,29.080
1,21.10,21.30,20.81,21.59,22.13,22.33,21.73,22.64,34.27,34.67,...,0.2668,0.2796,0,0,0.02427,0.02817,28.750,28.850,28.400,29.070


In [12]:
test = test_df.to_numpy()
test_size, feature_size = test.shape
# 因為 test 資料已經事先切割好範圍，故需要明確切分每段資料
test_size = test_size//input_date_data_size
test_x = np.empty([test_size, feature_size * input_date_data_size*power], dtype = float)

for idx in range(test_size):
  temp_data = np.array([])
  for count in range(input_date_data_size):
    temp_data = np.hstack([temp_data, test[idx * input_date_data_size + count]])
    for nth_term in range(2,power+1):
        temp_data = np.hstack([temp_data, test[idx * input_date_data_size + count]**nth_term])
  test_x[idx, :] = temp_data

# test 資料也需要照 training 方式做正規化
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]

In [13]:
test_x = torch.from_numpy(test_x.astype(np.float32))
predicted = model(test_x)

ids = [x for x in range(len(predicted))]
output_df = pd.DataFrame({'id': ids})
# 要按照規定順序設定col
currency_columns = ["AUD", "CAD", "EUR", "GBP", "HKD", "JPY", "KRW", "USD"]

for i, column_name in enumerate(currency_columns):
    output_df[column_name] = [x[i] for x in predicted.tolist()]


output_df.to_csv('./submission.csv', index=False)